In [27]:
from dolfinx import mesh as msh
from mpi4py import MPI
from dolfinx import fem
from dolfinx import default_scalar_type
import ufl

def pde_data(domain):
    Wh = fem.functionspace(domain, ('P', 2))
    domain.topology.create_connectivity(domain.topology.dim - 1, domain.topology.dim)
    boundary_facets = msh.exterior_facet_indices(domain.topology)
    boundary_dofs = fem.locate_dofs_topological(Wh, domain.topology.dim-1, boundary_facets)
    eps = fem.Constant(domain, default_scalar_type(1e-8))
    b = ufl.as_vector((fem.Constant(domain, default_scalar_type(1.0)),fem.Constant(domain, default_scalar_type(0.0))))
    c = fem.Constant(domain, default_scalar_type(0.0))
    f = fem.Constant(domain, default_scalar_type(1.0))
    x = ufl.SpatialCoordinate(domain)
    expr = x[0]
    u_exact = fem.Expression(expr, Wh.element.interpolation_points())
    uD = fem.Function(Wh)
    uD.interpolate(u_exact)
    bcs = [fem.dirichletbc(uD, boundary_dofs)]
    return Wh,eps,b,c,f,bcs,None

In [28]:
import pyvista as pv
from supg import supg
from dolfinx import mesh as msh
from mpi4py import MPI

domain = msh.create_unit_square(MPI.COMM_WORLD, 16, 16, msh.CellType.triangle)

sd = supg.data(domain=domain, pde_data=pde_data, boundary_eval=False)
sd.set_weights(1e-2)
stg = sd.create_stage()

weights = sd.yh.x.array


p = pv.Plotter()


stg.add_data(sd.uh)
p.add_mesh(stg.grid, show_edges=True)

p.show()

Widget(value='<iframe src="http://localhost:49546/index.html?ui=P_0x353c5bd90_18&reconnect=auto" class="pyvist…

In [ ]:
from dolfinx import mesh as msh
from mpi4py import MPI
from dolfinx import fem
from dolfinx import default_scalar_type
import ufl
import numpy as np
import torch

x = torch.load('data/example_3_1/training_set/inputs/x_1.pt')

def tensor_to_fem_data(x):
    bK = x[9:11, 1,1]
    if torch.allclose(bK, torch.tensor([ 1.0, 0.0], dtype = bK.dtype)):
        b = ufl.as_vector((fem.Constant(domain, default_scalar_type(1.0)),fem.Constant(domain, default_scalar_type(0.0))))
        dir = 0

    if torch.allclose(bK, torch.tensor([ -1.0, 0.0], dtype = bK.dtype)):
        b = ufl.as_vector((fem.Constant(domain, default_scalar_type(-1.0)),fem.Constant(domain, default_scalar_type(0.0))))
        dir = 0

    if torch.allclose(bK, torch.tensor([ 0.0, 1.0], dtype = bK.dtype)):
        b = ufl.as_vector((fem.Constant(domain, default_scalar_type(0.0)),fem.Constant(domain, default_scalar_type(1.0))))
        dir = 1

    if torch.allclose(bK, torch.tensor([ 0.0, -1.0], dtype = bK.dtype)):
        b = ufl.as_vector((fem.Constant(domain, default_scalar_type(0.0)),fem.Constant(domain, default_scalar_type(-1.0))))
        dir = 1


    ex_exp = x[dir]*(1-ufl.exp(-(1-x[dir])/eps))* (1 - (((ufl.exp(-(1-x[(dir+1)%2])/eps)  + ufl.exp(-(x[(dir+1)%2])/eps)))- ufl.exp(-(1)/eps))/(1-ufl.exp(-1/eps)))

    if torch.min(bK) < 0:
        ex_exp = (1-x[dir])*(1-ufl.exp(-(x[dir])/eps))* (1 - (((ufl.exp(-(1-x[(dir+1)%2])/eps)  + ufl.exp(-(x[(dir+1)%2])/eps)))- ufl.exp(-(1)/eps))/(1-ufl.exp(-1/eps)))


        
    
    uD = torch.concat((x[12,0,:],x[12,-1,:],x[12,:,0],x[12,:,-1]))
    if torch.allclose(max(uD), torch.tensor([ 0.0], dtype = bK.dtype)):
        ex_exp = x[dir]
    if np.min(bK) < 0:
        ex_exp = 1 - ex_exp

    #Ex 1
def pde_data1(domain):
    Wh = fem.functionspace(domain, ('P', 2))
    domain.topology.create_connectivity(domain.topology.dim - 1, domain.topology.dim)
    boundary_facets = msh.exterior_facet_indices(domain.topology)
    boundary_dofs = fem.locate_dofs_topological(Wh, domain.topology.dim-1, boundary_facets)
    eps = fem.Constant(domain, default_scalar_type(1e-8))
    c = fem.Constant(domain, default_scalar_type(0.0))
    f = fem.Constant(domain, default_scalar_type(1.0))
    bcs = [fem.dirichletbc(fem.Constant(domain, default_scalar_type(0.0)), boundary_dofs, Wh)]
    return Wh,eps,b,c,f,bcs,None

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=torch.float64)

In [38]:
if torch.min(bK) < 0:
    print(2)

2
